<a href="https://colab.research.google.com/github/MarioKingII/CSE210.1/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import XGBoost, other libraries, and read the data

In [ ]:
# Import XGBoost and other libraries
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn import linear_model
# Read Data
data = pd.read_csv("snp500_history.csv")

In [ ]:
data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1927-12-30 00:00:00-05:00,17.66,17.66,17.66,17.66,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1928-01-03 00:00:00-05:00,17.76,17.76,17.76,17.76,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean up the data

In [ ]:

data = data[data['Volume'] != 0]
data.drop('Dividends', axis=1, inplace=True)
data.drop('Stock Splits', axis=1, inplace=True)

data['Date'] = pd.to_datetime(data['Date'], utc=True)
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

data = data[data['Year'] >= 1983]

data['Next Year Close'] = data['Close'].shift(-252)

data.dropna(inplace=True)


data.sort_values('Date', inplace=True)


In [ ]:
print(data)
data.to_csv(r"modified_data.csv", index=False)


                           Date         Open         High          Low  \
27563 1983-01-03 05:00:00+00:00   140.649994   141.330002   138.199997   
27565 1983-01-04 05:00:00+00:00   138.330002   141.360001   138.080002   
27567 1983-01-05 05:00:00+00:00   141.350006   142.600006   141.149994   
27569 1983-01-06 05:00:00+00:00   142.009995   145.770004   142.009995   
27571 1983-01-07 05:00:00+00:00   145.270004   146.460007   145.149994   
...                         ...          ...          ...          ...   
47427 2022-05-25 04:00:00+00:00  3929.590088  3999.330078  3925.030029   
47429 2022-05-26 04:00:00+00:00  3984.600098  4075.139893  3984.600098   
47431 2022-05-27 04:00:00+00:00  4077.429932  4158.490234  4077.429932   
47433 2022-05-31 04:00:00+00:00  4151.089844  4168.339844  4104.879883   
47435 2022-06-01 04:00:00+00:00  4149.779785  4166.540039  4073.850098   

             Close        Volume    Year  Month   Day  Next Year Close  
27563   138.339996  5.908000e+07  1983

In [ ]:
# Split Data into X and Y
X = data.drop(['Date', 'Next Year Close', 'Year', 'Month', 'Day'], axis=1)

# Log transformation for Y
log_target = np.log(data["Next Year Close"])
y = log_target

In [ ]:
# Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

y_train.isnull().sum()
#Fit Model to Training data
model = xgb.XGBRegressor(n_estimators=1000,
                         max_depth=7,
                         eta=0.1,
                         subsample=0.7,
                         colsample_bytree=0.8)

model.fit(X_train, y_train)

# Make Predicitons
y_pred = model.predict(X_test)


In [ ]:
nan_values = data[data['Next Year Close'].isnull()]
print(nan_values)

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Volume, Year, Month, Day, Next Year Close]
Index: []


In [ ]:

exp_test = np.exp(y_test)
exp_pred = np.exp(y_pred)

rmse = mean_squared_error(exp_test, exp_pred, squared=True)
print("Mean Squared Error:", rmse)

root = mean_squared_error(exp_test, exp_pred, squared=False)
print("Root Mean Squared Error:", root)

r2 = r2_score(exp_test, exp_pred)
print("R-squared score:", r2)



Mean Squared Error: 21675.10585205059
Root Mean Squared Error: 147.22467813532685
R-squared score: 0.9814348336289389


In [ ]:


prediction_data = pd.read_csv("Prediction_data.csv")
prediction_data_copy = prediction_data.copy()

# Drop the unnecessary columns
prediction_data.drop(['Date'], axis=1, inplace=True)
prediction_data.drop(['Month'], axis=1, inplace=True)
prediction_data.drop(['Day'], axis=1, inplace=True)
prediction_data.drop(['Year'], axis=1, inplace=True)

prediction_data.head()


,Open,High,Low,Close,Volume
0,4095.409912,4177.509766,4074.370117,4176.819824,4405790000
1,4137.569824,4142.669922,4098.669922,4108.540039,3711110000
2,4134.720215,4168.779785,4109.180176,4121.430176,4332700000
3,4096.470215,4164.859863,4080.189941,4160.680176,4248210000
4,4147.120117,4160.140137,4107.200195,4115.770020,4159470000


In [ ]:
# Make predictions on the new data
predicted_close = model.predict(prediction_data)

# Convert the predicted log-transformed values back to the original scale
predicted_close_original_scale = np.exp(predicted_close)

print("Predicted next year closing prices:")
for i, price in enumerate(predicted_close_original_scale):
    print(f"Date: {prediction_data_copy['Date'][i]}, Predicted Next Year Close: {price:.2f}")


Predicted next year closing prices:
Date: 2022-06-02 04:00:00+00:00, Predicted Next Year Close: 4160.75
Date: 2022-06-03 04:00:00+00:00, Predicted Next Year Close: 4334.17
Date: 2022-06-06 04:00:00+00:00, Predicted Next Year Close: 4182.87
Date: 2022-06-07 04:00:00+00:00, Predicted Next Year Close: 4058.14
Date: 2022-06-08 04:00:00+00:00, Predicted Next Year Close: 4235.31
Date: 2022-06-09 04:00:00+00:00, Predicted Next Year Close: 4246.70
Date: 2022-06-10 04:00:00+00:00, Predicted Next Year Close: 4272.82
Date: 2022-06-13 04:00:00+00:00, Predicted Next Year Close: 4459.06
Date: 2022-06-14 04:00:00+00:00, Predicted Next Year Close: 4668.01
Date: 2022-06-15 04:00:00+00:00, Predicted Next Year Close: 4428.66
Date: 2022-06-16 04:00:00+00:00, Predicted Next Year Close: 4615.71
Date: 2022-06-17 04:00:00+00:00, Predicted Next Year Close: 4550.43
Date: 2022-06-21 04:00:00+00:00, Predicted Next Year Close: 4674.78
Date: 2022-06-22 04:00:00+00:00, Predicted Next Year Close: 4625.55
Date: 2022-0